<a href="https://colab.research.google.com/github/SravaniLoka/House-Price-predictor-/blob/main/Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image recognition using Python,OpenCV,OpenAI CLIP model and POstgreSQL pgvector


# STEP 0: INSTALL REQUIREMENTS

In [1]:
!pip install opencv-python imgbeddings psycopg2-binary

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.5 MB/s eta 0:00:00
  Created wheel for imgbeddings: filename=imgbeddings-0.1.0-py3-none-any.whl size=9320 sha256=278fb2ba4dee6761f16ccc38564e8fad5ed24c50e9771bc573bb9f045f8738ee
  Stored in directory: /root/.cache/pip/wheels/4e/c9/9c/5f94fd783ce863540e5999f4bc2afd1501ddaea942ee6413fc
Successfully built imgbeddings


# STEP 1: FACE RECOGNITION

In [2]:
# importing the cv2 library
import cv2

# loading the haar case algorithm file into alg variable
alg = "/content/haarcascade_frontalface_default.xml"
# passing the algorithm to OpenCV
haar_cascade = cv2.CascadeClassifier(alg)
# loading the image path into file_name variable - replace <INSERT YOUR IMAGE NAME HERE> with the path to your image
file_name = "/content/modern-family-dvd-poster-cover-qlhbavon5k0kmbhn.jpg"
# reading the image
img = cv2.imread(file_name, 0)
# creating a black and white version of the image
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
# detecting the faces
faces = haar_cascade.detectMultiScale(
    gray_img, scaleFactor=1.05, minNeighbors=2, minSize=(100, 100)
)

i = 0
# for each face detected
for x, y, w, h in faces:
    # crop the image to select only the face
    cropped_image = img[y : y + h, x : x + w]
    # loading the target image path into target_file_name variable  - replace <INSERT YOUR TARGET IMAGE NAME HERE> with the path to your target image
    target_file_name = '/content/stored faces' + str(i) + '.jpg'
    cv2.imwrite(
        target_file_name,
        cropped_image,
    )
    i = i + 1;

# STEP 2: EMBEDDINGS CALCULATION


In [1]:
# importing the required libraries
import numpy as np
from imgbeddings import imgbeddings
from PIL import Image
import psycopg2
import os

# connecting to the database - replace the SERVICE URI with the service URI
conn = psycopg2.connect("<SERVICE_URI>")

for filename in os.listdir("/content/stored faces"):
    # opening the image
    img = Image.open("/content/stored faces" + filename)
    # loading the `imgbeddings`
    ibed = imgbeddings()
    # calculating the embeddings
    embedding = ibed.to_embeddings(img)
    cur = conn.cursor()
    cur.execute("INSERT INTO pictures values (%s,%s)", (filename, embedding[0].tolist()))
    print(filename)
conn.commit()

ModuleNotFoundError: No module named 'imgbeddings'

# STEP 3:CALCULATE EMBEDDINGS ON NEW PICTURE


In [ ]:
# loading the face image path into file_name variable
file_name = "solo-image.png"  # replace <INSERT YOUR FACE FILE NAME> with the path to your image
# opening the image
img = Image.open(file_name)
# loading the `imgbeddings`
ibed = imgbeddings()
# calculating the embeddings
embedding = ibed.to_embeddings(img)

STEP 3:FIND SIMILAR IMAGES BY QUERYING THE POSTGRESQL DB USING PGVECTOR

In [ ]:
from IPython.display import Image, display

cur = conn.cursor()
string_representation = "["+ ",".join(str(x) for x in embedding[0].tolist()) +"]"
cur.execute("SELECT * FROM pictures ORDER BY embedding <-> %s LIMIT 1;", (string_representation,))
rows = cur.fetchall()
for row in rows:
    display(Image(filename="stored-faces/"+row[0]))
cur.close()